In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [18]:
tatamotors = yf.download('TATAMOTORS.BO')
titan = yf.download('TITAN.BO')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [19]:
tatamotors.tail(5)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-06-14,991.000000,997.000000,981.400024,993.400024,993.400024,404491
2024-06-18,998.700012,1003.450012,984.099976,985.650024,985.650024,351324
2024-06-19,990.900024,994.900024,975.549988,977.400024,977.400024,312686
2024-06-20,981.200012,988.099976,976.750000,978.099976,978.099976,532805
2024-06-21,979.900024,981.000000,958.000000,961.049988,961.049988,905646


In [20]:
import pandas_ta as ta
def GCSignal(df):
    df['SMA10']=ta.sma(df.Close,length=10)
    df['SMA50'] = ta.sma(df.Close,length=50)
    df['buySignal']=np.where(df.SMA10>df.SMA50,1,0)
    df['sellSignal']=np.where(df.SMA10<df.SMA50,1,0)
    df['Decision Buy GC']= df.buySignal.diff()
    df['Decision Sell GC']= df.sellSignal.diff()

In [21]:
GCSignal(tatamotors)
GCSignal(titan)

In [22]:
tatamotors

,Open,High,Low,Close,Adj Close,Volume,SMA10,SMA50,buySignal,sellSignal,Decision Buy GC,Decision Sell GC
Date,,,,,,,,,,,,
2000-01-03,43.500000,43.500000,43.500000,43.500000,16.015545,0,NaN,NaN,0,0,NaN,NaN
2000-01-04,43.500000,43.500000,43.500000,43.500000,16.015545,0,NaN,NaN,0,0,0.0,0.0
2000-01-05,43.500000,43.500000,43.500000,43.500000,16.015545,0,NaN,NaN,0,0,0.0,0.0
2000-01-06,43.500000,43.500000,43.500000,43.500000,16.015545,0,NaN,NaN,0,0,0.0,0.0
2000-01-07,43.500000,43.500000,43.500000,43.500000,16.015545,0,NaN,NaN,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-14,991.000000,997.000000,981.400024,993.400024,993.400024,404491,962.275006,979.992000,0,1,0.0,0.0
2024-06-18,998.700012,1003.450012,984.099976,985.650024,985.650024,351324,965.790009,979.619000,0,1,0.0,0.0
2024-06-19,990.900024,994.900024,975.549988,977.400024,977.400024,312686,973.115009,978.986001,0,1,0.0,0.0


In [24]:
if tatamotors['Decision Sell GC'].iloc[-1]==1:
    print("Sell Signal with Golden crossover for the stock")
if tatamotors['Decision Buy GC'].iloc[-1]==1:
    print("Buy Signal with Golden crossover for the stock")

Buy Signal with Golden crossover for the stock


### BACKTESTING THE CROSSOVER STRATEGY 

In [25]:
bi,si=[],[]
Buy, Sell=[],[]
Buyp,Sellp=[],[]
Flag = False
for i in range(1,len(tatamotors)):
    if not Flag:
        if tatamotors.SMA10.iloc[i] > tatamotors.SMA50.iloc[i]  and tatamotors.SMA10.iloc[i-1] < tatamotors.SMA50.iloc[i-1]:
            bi.append(i+1)
            Buy.append(tatamotors.iloc[i+1].name)
            Buyp.append(tatamotors.iloc[i+1].Open)
            Flag=True
    if Flag:
        if tatamotors.SMA10.iloc[i] < tatamotors.SMA50.iloc[i]  and tatamotors.SMA10.iloc[i-1] > tatamotors.SMA50.iloc[i-1]:
            si.append(i+1)     
            Sell.append(tatamotors.iloc[i+1].name)
            Sellp.append(tatamotors.iloc[i+1].Open)
            Flag=False

IndexError: single positional indexer is out-of-bounds